## Imports and Garbage Collection

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from brightest_path_lib.algorithm import AStarSearch # type: ignore
from brightest_path_lib.input import CostFunction, HeuristicFunction # type: ignore
from brightest_path_lib.cost import ReciprocalConsistency, ConsistencyTurns
from IPython.display import Image
import json
import matplotlib.image as mpimg
import cv2
import time
import gc

gc.collect()

16

## Settings

In [2]:
img_file = "images/ex1.png"
cost_func = "ct-wght-sum_rgb-thresh"
parameters = "dir-wght=10000000"
bbox_file = "images/ex1_paths.json"
# start_end_dict = {"pair1": [(2270, 4246), (5117, 4600)],
#                   "pair2": [(397, 1137), (700, 7303)],
#                   "pair3": [(3026, 1571), (5128, 3577)],
#                   "pair4": [(4158, 5742), (5513, 5860)],
#                   "pair5": [(5100, 941), (3021, 1216)],
#                   }
# pair = "pair3"
# start_pixel, end_pixel = start_end_dict[pair][0], start_end_dict[pair][1]


## Pre-Processing

##### Importing Bounding Boxes

In [3]:
bbox_json = {}
with open(bbox_file) as file:
    text = file.read()
    bbox_json = json.loads(text)
bbox_json.items()

dict_items([('Image filename', 'ex1.png'), ('Image width', 2714), ('Image height', 656), ('Line', [{'type': 'path', 'x0': 169.43, 'y0': 288.27, 'x1': 457.87, 'y1': 282.37}, {'type': 'path', 'x0': 1199.47, 'y0': 378.3, 'x1': 1199.54, 'y1': 477.91}])])

In [4]:
bbox_json = {}
with open(bbox_file) as file:
    text = file.read()
    bbox_json = json.loads(text)

start_bbox = []
end_bbox = []
count = 0

for _, val in bbox_json.items():
    if type(val) == list:
        while count < 2:
            box_dict = val[count]
            if box_dict["type"] == "path" and count == 0:
                count += 1
                start_bbox.append(int(box_dict["x0"]))
                start_bbox.append(int(box_dict["y0"]))
                start_bbox.append(int(box_dict["x1"]))
                start_bbox.append(int(box_dict["y1"]))
            elif box_dict["type"] == "path" and count == 1:
                count += 1
                end_bbox.append(int(box_dict["x0"]))
                end_bbox.append(int(box_dict["y0"]))
                end_bbox.append(int(box_dict["x1"]))
                end_bbox.append(int(box_dict["y1"]))

print(start_bbox, end_bbox)


[169, 288, 457, 282] [1199, 378, 1199, 477]


##### Image Processing

In [5]:
img_rgba = mpimg.imread(img_file)
img_gray = 1 - cv2.cvtColor(img_rgba, cv2.COLOR_BGR2GRAY)  # invert color

##### Finding Possible Starting and Ending Points

In [6]:
def traverse_yedge(img_gray, main_axis, sec_axis_start, sec_axis_end):
    """
    TODO: write docstring
    """
    intensity_centers = []
    MIN_INTENSITY = 0.1
    curr_int_range = []
    y = main_axis
    for x in range(sec_axis_start, sec_axis_end):
        if img_gray[y, x] > MIN_INTENSITY:
            curr_int_range.append(x)
        if len(curr_int_range) > 0 and img_gray[y, x] <= MIN_INTENSITY:
            # TODO: change x to curr_int_range[-1]
            mid_point = int((x + curr_int_range[0]) / 2)
            intensity_centers.append((mid_point, y))
            curr_int_range = []
    return intensity_centers

def traverse_xedge(img_gray, main_axis, sec_axis_start, sec_axis_end):
    """
    TODO: write docstring
    """
    intensity_centers = []
    MIN_INTENSITY = 0.1
    curr_int_range = []
    x = main_axis
    for y in range(sec_axis_start, sec_axis_end):
        if img_gray[y, x] > MIN_INTENSITY:
            curr_int_range.append(y)
        if len(curr_int_range) > 0 and img_gray[y, x] <= MIN_INTENSITY:
            # TODO: change x to curr_int_range[-1]
            mid_point = int((y + curr_int_range[0]) / 2)
            intensity_centers.append((x, mid_point))
            curr_int_range = []
    return intensity_centers

In [7]:
# img_gray, start_bbox, end_bbox
def get_intensity_centers(img_gray, bbox):
    x0 = bbox[0]
    y0 = bbox[1]
    x1 = bbox[2]
    y1 = bbox[3]

    intensity_centers = []
    intensity_centers += traverse_xedge(img_gray, x0, y0, y1)
    intensity_centers += traverse_xedge(img_gray, x1, y0, y1)
    intensity_centers += traverse_yedge(img_gray, y0, x0, x1)
    intensity_centers += traverse_yedge(img_gray, y1, x0, x1)
    return intensity_centers

start_points = get_intensity_centers(img_gray, start_bbox)
end_points = get_intensity_centers(img_gray, end_bbox)
print(f"End Points: {end_points}")
print(f"Start Points: {start_points}")
print(start_bbox, end_bbox)
end_points[1:2]

End Points: []
Start Points: [(179, 288), (287, 288), (359, 288), (188, 282), (287, 282)]
[169, 288, 457, 282] [1199, 378, 1199, 477]


[]

##### Cost Function and Search Algorithm Definitions

In [ ]:
astar_results = {}
for start_pt in start_points:
    start_pixel = (start_pt[1], start_pt[0])
    consistency_cost_func: CostFunction = ReciprocalConsistency(0, 1, img_rgba[start_pixel])
    if cost_func[:2] == "ct":
        consistency_cost_func: CostFunction = ConsistencyTurns(0, 1, img_rgba[start_pixel])
    for end_pt in end_points:
        end_pixel = (end_pt[1], end_pt[0])
        gc.collect()
        astar = AStarSearch(img_gray, img_rgba, start_pixel, end_pixel,
                            cost_function=consistency_cost_func)
        astar_start_time = time.time()
        print(f"Starting AStar on {start_pixel} to {end_pixel}...")
        path = astar.search()
        astar_end_time = time.time()
        print("AStar ended.")
        print(f"Astar Run Time: {astar_end_time - astar_start_time} seconds")
        if astar is None or astar.result == []: 
            print(f"AStar failed to find path from {start_pixel} to {end_pixel}.")
        else:
            astar_results[(start_pixel, end_pixel)] = (astar.result, path)

## Testing

In [ ]:
# print(img_rgba[(402, 1075)])
# print(img_rgba[(397, 1137)])
# print(img_rgba[(701, 7297)])
# print(img_rgba[(700, 7303)])
# astar_results

In [ ]:
# start_pixel, end_pixel = ((402, 1075), (701, 7297))
# start_pixel, end_pixel = ((397, 1137), (700, 7303))
start_pixel, end_pixel = ((3026, 1571), (5128, 3577))
consistency_cost_func: CostFunction = ReciprocalConsistency(0, 1, img_rgba[start_pixel])
if cost_func[:2] == "ct":
    consistency_cost_func: CostFunction = ConsistencyTurns(0, 1, img_rgba[start_pixel])

astar = AStarSearch(img_gray, img_rgba, start_pixel, end_pixel,
                    cost_function=consistency_cost_func)
astar_start_time = time.time()
print(f"Starting AStar on {start_pixel} to {end_pixel}...")
path = astar.search()
astar_end_time = time.time()
print("AStar ended.")
print(f"Astar Run Time: {astar_end_time - astar_start_time} seconds")


## Plotting Result

In [ ]:
for points, result_tup in astar_results.items():
    start_pixel = points[1]
    end_pixel = points[0]
    result, path = result_tup
    print("Loading Image ...")
    plt.imshow(img_rgba)
    print("Plotting Points and Path ...")
    plt.plot(start_pixel[1], start_pixel[0], 'og')
    plt.plot(end_pixel[1], end_pixel[0], 'or')
    plt.plot([point[1] for point in result], [point[0] for point in path], '-b', linewidth=3)
    plt.plot(start_pixel[1], start_pixel[0], 'og')
    plt.plot(end_pixel[1], end_pixel[0], 'or')
    plt.tight_layout()
    print("Saving Figure ...")
    fig_name = f"images/substation3_200_{points[0]}_to_{points[1]}_demo.png"
    plt.savefig(fig_name)
    plt.close()
    print('Done')

## Show Image

In [ ]:
# Image(filename=fig_name)

## Animation

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")

In [ ]:
%%capture
# astar.result
import matplotlib.animation as animation

# fig, ax = plt.subplots(figsize=(60,84))
fig, ax = plt.subplots(figsize=(5,7))
img_fig = ax.imshow(img_rgba)
plt.plot(start_pixel[1], start_pixel[0], 'og', markersize=5)
plt.plot(end_pixel[1], end_pixel[0], 'or', markersize=5)


In [ ]:
def plot_frame_func(frame, ax, points):
    # ax.clear()
    ax.plot(points[frame][1], points[frame][0], 'bo', markersize=3)
    return ax

In [ ]:
# frames_indicies = range(len(astar.result))
# %matplotlib widget
total_frames = len(astar.result)
step = int(total_frames / 30)
frames_indicies = range(0, total_frames, step)

ani = animation.FuncAnimation(fig, plot_frame_func, frames_indicies,
                              interval=100, 
                              repeat_delay=5000, blit=False,
                              fargs=(ax, astar.result))

# plt.show()

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
print("Saving animation ...")
ani.save(f"images/animation_{pair}_{cost_func}_{parameters}.mp4")

In [ ]:
img_rgba.shape
astar.result[0][1]
len(astar.result)